In [8]:
# Commands to run in the terminal to install the required packages
""" 
conda create -n ollama python=3.10.9
conda activate ollama
pip install -r requirements.txt
"""
# ! pip install ollama 

' \nconda create -n ollama python=3.10.9\nconda activate ollama\npip install -r requirements.txt\n'

In [1]:
# Lets see how many model are included
import ollama
ollama.list()['models']

[{'name': 'llama2:latest',
  'model': 'llama2:latest',
  'modified_at': '2024-04-14T13:46:30.5027781+02:00',
  'size': 3826793677,
  'digest': '78e26419b4469263f75331927a00a0284ef6544c1975b826b15abdaef17bb962',
  'details': {'parent_model': '',
   'format': 'gguf',
   'family': 'llama',
   'families': ['llama'],
   'parameter_size': '7B',
   'quantization_level': 'Q4_0'}},
 {'name': 'mistral:latest',
  'model': 'mistral:latest',
  'modified_at': '2024-04-14T14:02:32.2995197+02:00',
  'size': 4109865159,
  'digest': '61e88e884507ba5e06c49b40e6226884b2a16e872382c2b44a42f2d119d804a5',
  'details': {'parent_model': '',
   'format': 'gguf',
   'family': 'llama',
   'families': ['llama'],
   'parameter_size': '7B',
   'quantization_level': 'Q4_0'}}]

## Test with "ollama2"

In [2]:
# Pull llama2 model
ollama.pull("llama2")
ollama.show("llama2")

{'license': 'LLAMA 2 COMMUNITY LICENSE AGREEMENT\t\nLlama 2 Version Release Date: July 18, 2023\n\n"Agreement" means the terms and conditions for use, reproduction, distribution and \nmodification of the Llama Materials set forth herein.\n\n"Documentation" means the specifications, manuals and documentation \naccompanying Llama 2 distributed by Meta at ai.meta.com/resources/models-and-\nlibraries/llama-downloads/.\n\n"Licensee" or "you" means you, or your employer or any other person or entity (if \nyou are entering into this Agreement on such person or entity\'s behalf), of the age \nrequired under applicable laws, rules or regulations to provide legal consent and that \nhas legal authority to bind your employer or such other person or entity if you are \nentering in this Agreement on their behalf.\n\n"Llama 2" means the foundational large language models and software and \nalgorithms, including machine-learning model code, trained model weights, \ninference-enabling code, training-en

In [3]:
# Set model to llama2 and chat with it
stream = ollama.chat(
    model='llama2',
    messages=[{'role': 'user', 'content': 'What is Flagger Force?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Flagger Force is a workforce solutions provider that specializes in flagging and traffic control services for construction and infrastructure projects. The company was founded in 1976 and has since grown to become one of the largest providers of flagging and traffic control services in the United States.

Flagger Force provides trained flaggers and traffic controllers to assist with traffic flow management, safety monitoring, and other related services for construction sites, roadway projects, and special events. Their services include flagging, traffic control, and safety monitoring, as well as training and consulting services for clients in various industries.

Flagger Force operates in several states across the United States, including California, Texas, Florida, and New York, among others. The company is committed to providing high-quality services that meet or exceed client expectations, while also ensuring the safety of both their employees and the general public.

Overall, Flag

## Test with "mistral"

In [4]:
# Pull mistral model
ollama.pull("mistral")
ollama.show("mistral")

{'license': '                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION\n\n   1. Definitions.\n\n      "License" shall mean the terms and conditions for use, reproduction,\n      and distribution as defined by Sections 1 through 9 of this document.\n\n      "Licensor" shall mean the copyright owner or entity authorized by\n      the copyright owner that is granting the License.\n\n      "Legal Entity" shall mean the union of the acting entity and all\n      other entities that control, are controlled by, or are under common\n      control with that entity. For the purposes of this definition,\n      "control" means (i) the power, direct or indirect, to cause the\n      direction or management of such entity, whether by contract or\n      otherwise, or (ii) ownership of fifty percent (50%) or more of the\n      outstanding

In [5]:
# Set model to mistral and chat with it
stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'What is Flagger Force?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 Flagger Force is not a widely known term and could refer to different things depending on the context. Here are some possibilities:

1. A children's book series: "Flagger Force" is a series of children's books written by Jeanne Willis and illustrated by Tony Ross. The stories follow the adventures of a team of traffic-controlling animals called the Flagger Force.
2. A traffic management company: Flagger Force could also be the name of a traffic management or construction company that provides flagging services to control traffic during roadwork or construction projects.
3. A military term: In military terminology, a "flagger force" is a group of soldiers responsible for controlling and managing traffic on the battlefield. This could include setting up checkpoints, directing vehicles and personnel, and ensuring the safe movement of troops and supplies.
4. A group of people who wave flags: In a more general sense, a "flagger force" could be a group of people who wave flags as a form of 

## RAG Implementation

In [6]:
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = WebBaseLoader(    
    web_paths=("https://flaggerforce.com/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("page-content", "page-body", "page-header")
        )
    ),
)
docs = loader.load()
docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits

[Document(page_content='Tap To Call\nLocations\nOrder Traffic Control\n\n\n\n\n\n\n\n\n\n  Menu  \n\nAbout Us\n\nService Areas\nCapabilities\nLeadership\nTestimonials\nKeeping Our Communities Moving\nPressroom\n\n\nBlog\nCareers\n\nApply for Field Positions\nApply for Office Positions\nWhat Is It Like to Work Here?\n\n\nContact Us\n\nQuestions & Feedback\nShow Appreciation\nService Areas\n\n\nORDER TRAFFIC CONTROL\n\n\n\n\nOrder Traffic Control\n\n\n\n About Us\nCapabilities\nBlog\nCareers\nEmployee Portal\nContact Us\nOrder Traffic Control\n \n \n\n\n\n\n\n\n\n\n\n  Employee Portal\n\n\n 888.312.3524\nLocations\nWe’re Hiring Crew Members\n \n\n\nAbout Us\n\nService Areas\nCapabilities\nLeadership\nTestimonials\nKeeping Our Communities Moving\nPressroom\n\n\nBlog\nCareers\n\nApply for Field Positions\nApply for Office Positions\nWhat Is It Like to Work Here?\n\n\nContact Us\n\nQuestions & Feedback\nShow Appreciation\nService Areas\n\n\nORDER TRAFFIC CONTROL', metadata={'source': 'https

In [7]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Create the retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

# Use the RAG chain
result = rag_chain("What is Flagger Force?")
print(result)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


 Flagger Force is a traffic control services company. Based on the information provided in the context, it appears that they offer orderly traffic control services for various projects and events. They have capabilities in both field positions and office positions, and they have an employee portal for their staff. Their contact details are also available for questions, feedback, or showing appreciation.


In [8]:
# Use the RAG chain
result = rag_chain("How many trucks has Flagger Force?")
print(result)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


 I apologize, but the information about the number of trucks Flagger Force has is not readily available on their website. You may consider contacting them through their "Contact Us" page or calling their toll-free number (888.312.3524) to inquire about that specific detail.


In [31]:
from bs4 import BeautifulSoup
import requests
import pprint

url = 'https://flaggerforce.com/blog'
html_doc = requests.get(url)
pprint.pprint(html_doc.content)

soup = BeautifulSoup(html_doc.content, 'html.parser')


(b'<!DOCTYPE html>\n<html class="no-js">\n\n<head>\n\t<meta charset="utf-8">'
 b'\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta name="vie'
 b'wport" content="width=device-width, initial-scale=1">\n\t<meta name=\'robot'
 b"s' content='index, follow, max-image-preview:large, max-snippet:-1, max-vide"
 b"o-preview:-1' />\n\n\t<!-- This site is optimized with the Yoast SEO plugin"
 b' v21.9.1 - https://yoast.com/wordpress/plugins/seo/ -->\n\t<title>Blog - T'
 b'raffic Control Resources | Flagger Force</title>\n\t<link rel="canonical" '
 b'href="https://flaggerforce.com/blog/" />\n\t<link rel="next" href="https:/'
 b'/flaggerforce.com/blog/page/2/" />\n\t<meta property="og:locale" content="'
 b'en_US" />\n\t<meta property="og:type" content="article" />\n\t<meta property'
 b'="og:title" content="Blog - Traffic Control Resources | Flagger Force" /'
 b'>\n\t<meta property="og:url" content="https://flaggerforce.com/blog/" />\n\t'
 b'<meta property="og:site_name" content="Fla

In [32]:
soup.select('p')

[<p>Every year, the United States observes National Work Zone Awareness Week (NWZAW), a campaign to enhance safety in work zones. This initiative, which this year […]</p>,
 <p>As an industry-leading, Safety-Driven®, short-term traffic control company operating more than 1,000 active work zones across the East Coast daily, we understand the dangers of working on or near the …</p>,
 <p>
 Crew member Ricky Weller represents what Flagger Force seeks in its employees—hard-working, dependable, and safety-minded. Ricky has strengthened Flagger Force’s reputation as a premier traffic-control provider with each client he’s …</p>,
 <p>Every year, the United States observes National Work Zone Awareness Week (NWZAW), a campaign to enhance safety in work zones. This initiative, which this year runs from April 15 – …</p>,
 <p>The concept of “fair chance employment/hiring” is gaining momentum as a critical initiative in creating equitable opportunities for individuals with criminal records. Fair chan

In [14]:
# 1. split the data into chunks
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter

# urls = ["https://flaggerforce.com/",
#         "https://flaggerforce.com/blog/",
#         "https://flaggerforce.com/about-us/"]

urls = ["https://flaggerforce.com/"]
        
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100)
doc_splits = text_splitter.split_documents(documents=docs_list)
doc_splits

ImportError: Could not import tiktoken python package. This is needed in order to calculate max_tokens_for_prompt. Please install it with `pip install tiktoken`.